In [1]:
'''
Code for establishing baseline scores instructor intervention predictions for a portion of the MOOC 
annotation corpus. This code predicts only results for Task 2.1 


'''

'''
Google n gram corpus for idf, --- http://trimc-nlp.blogspot.sg/2013/04/tfidf-with-google-n-grams-and-pos-tags.html
https://news.ycombinator.com/item?id=698991

lexicon for first names -- https://stackoverflow.com/questions/20290870/improving-the-extraction-of-human-names-with-nltk?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
https://nlp.stanford.edu/software/CRF-NER.shtml
https://machinelearningmastery.com/clean-text-machine-learning-python/


add no of posts, comments, no. of posts_comments, no. of sentences, avg. no of sentences per post/comment, 
avg. no of words per post to features
thread length ftrs -- no. posts+comments, posts, comments, no of sentences in thread and no. of words (after removing stop words)

from svm classifer -- output weights of features--word, tfidf weight, svm weight. After classification we can see top 10 weights
add more data
fix cross validation

'''

import numpy as np
import os
import nltk
import pandas as pd
from io import StringIO
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
import operator
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
df = pd.read_csv('../data/21total_preprocessed.csv')
print(tensorflow.VERSION)

/Users/radhikanikam/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


1.4.0


Using TensorFlow backend.


In [2]:
'''
Preparing and printing the data
'''

del df['Task2.1']
print(df.head())
df2 = pd.read_csv('../data/nustotal_preprocessed.csv')
df = df.append(df2)

df['Categories'] = df['Categories'].factorize()[0]
#print((df.head(10)))
print(df['Categories'].value_counts())

   Unnamed: 0                                              Title  \
0           0                        Put the Essays in an e-book   
1           1  Need help with 'common sense' understanding of...   
2           2                            Quiz deadline confusion   
3           3                   HELP--Anyone with good internet?   
4           4  It is not my fault that your site can not with...   

          Course  Categories  \
0  amnhearth-002    resolves   
1  amnhearth-002  elaborates   
2  amnhearth-002    resolves   
3  amnhearth-002    resolves   
4  amnhearth-002    resolves   

                                           Post_text Comment_text  Post_count  \
0  During the recent Human Evolution: Past and Pr...          NaN           1   
1                                                NaN          NaN           0   
2  Today I discovered this course and decided to ...          NaN           3   
3  Hi, I have spent the last three hours trying t...          NaN         

In [3]:
#Removing Names

def extract_entities(text):
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'node'):
                print(chunk.node, ' '.join(c[0] for c in chunk.leaves()))

# from nltk.tag import StanfordNERTagger
# from nltk.tokenize import word_tokenize

# st = StanfordNERTagger('/usr/share/stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz',
# 					   '/usr/share/stanford-ner/stanford-ner.jar',
# 					   encoding='utf-8')

# text = 'While in France, Christine Lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.'

# tokenized_text = word_tokenize(text)
# classified_text = st.tag(tokenized_text)

# print(classified_text)

# nltk.download('words')
# for i in df['Post_text'].dropna():
#     print(i)
#     extract_entities(i)

In [4]:
categories = ['resolves','elaborates','social','requests']
courses = df['Course'].unique()

for course in courses:
    course_data = df.loc[df['Course']==course].dropna()
    post_text = course_data['Post_text'].tolist()
    course_train, course_val, course_test = np.split(course_data.sample(frac=1), [int(.6*len(course_data)), int(.8*len(course_data))])    
df = df.dropna()
#df_train, df_val, df_test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])    
df_train, df_val, y_train, y_val = train_test_split(df.loc[:,'Post_text':'Comment_count'],df['Categories'],test_size=0.40,stratify=df['Categories']) 
#print(df_val)
count_vect = CountVectorizer()
df_train_counts = count_vect.fit_transform(df_train['Post_text'])

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
df_train_tfidf = tfidf_vectorizer.fit(df_train['Post_text'])
text=df_train['Post_text'].tolist()
#print(text[0])
X = df_train_tfidf.transform([text[0]])
#print(tfidf_vectorizer.vocabulary_)
#print(X)
#words_list = list(tfidf_vectorizer.vocabulary_.keys())
#print(words_list)
#print(sorted(tfidf_vectorizer.vocabulary_,key=tfidf_vectorizer.vocabulary_.get, reverse=True))
new_dict={}
#print(tfidf_vectorizer.idf_)
for vocab, idf_score in zip((list(tfidf_vectorizer.vocabulary_.keys())),(tfidf_vectorizer.idf_)):
    #print(vocab,idf_score)
    if len(vocab)>3:
        new_dict[vocab] = idf_score
print(sorted(new_dict.items(),key=operator.itemgetter(1),reverse=True)[:30])

print((X.toarray()))
'''
vectorizer = TfidfVectorizer()

posts=df_train['Post_text'].dropna().tolist()
vectorizer.fit(posts)
#print(vectorizer.vocabulary_)
print(vectorizer.idf_)
vector = vectorizer.transform([posts[0]])
print(vector.shape)
print(vector.toarray())
'''
    

[('model', 4.3141860046725258), ('described', 4.3141860046725258), ('lectures', 4.3141860046725258), ('pane', 4.3141860046725258), ('glass', 4.3141860046725258), ('absorbs', 4.3141860046725258), ('radiation', 4.3141860046725258), ('layer', 4.3141860046725258), ('ground', 4.3141860046725258), ('escapes', 4.3141860046725258), ('directly', 4.3141860046725258), ('greenhouse', 4.3141860046725258), ('formed', 4.3141860046725258), ('matthew', 4.3141860046725258), ('really', 4.3141860046725258), ('ahead', 4.3141860046725258), ('features', 4.3141860046725258), ('splendid', 4.3141860046725258), ('francois', 4.3141860046725258), ('link', 4.3141860046725258), ('good', 4.3141860046725258), ('answer', 4.3141860046725258), ('space', 4.3141860046725258), ('works', 4.3141860046725258), ('believe', 4.3141860046725258), ('subject', 4.3141860046725258), ('culture', 4.3141860046725258), ('cindy', 4.3141860046725258), ('sherman', 4.3141860046725258), ('dialogue', 4.3141860046725258)]
[[ 0.  0.  0. ...,  0. 

"\nvectorizer = TfidfVectorizer()\n\nposts=df_train['Post_text'].dropna().tolist()\nvectorizer.fit(posts)\n#print(vectorizer.vocabulary_)\nprint(vectorizer.idf_)\nvector = vectorizer.transform([posts[0]])\nprint(vector.shape)\nprint(vector.toarray())\n"

In [5]:
# TO DO --  get tf-idf scores from google-ngram-corpus?

# from google_ngram_downloader import readline_google_store
# for word in words_list:
#     count = 0
#     fname, url, records = next(readline_google_store(ngram_len=1, indices=word[0]))
 
#     try:
#         record = next(records)

#         while record.ngram != word:
#             record = next(records)

#         while record.ngram == word:
#             count = count + record.match_count
#             record = next(records)

#     except StopIteration:
#         pass
#     print(count)
    

In [6]:
class_weights = {
    0: 1.,
    1: 1.,
    2: 5.,
    3: 5.
}


# Linear SVM or SGDCLassifier with Hinge Loss
print(" Linear SVM of SGDClassifier with Hinge Loss - Accuracy and feature names for categories")
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
                     ('tfidf', TfidfVectorizer(stop_words='english',ngram_range=(1,2))),
                     ('clf',  SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42 #class_weight=class_weights,
                                            )),
])
text_clf.fit(df_train['Post_text'], y_train)  

predicted = text_clf.predict(df_val['Post_text'])
print("Predicted: "+ str(predicted.tolist()))
print("Actual: " + str(y_val.tolist()))
print("Accuracy: "+ str(np.mean(predicted == y_val))+ "\n")
indices = (np.argsort(text_clf.named_steps['clf'].coef_,axis=1))
#print(indices)
features = (text_clf.named_steps['tfidf'].get_feature_names())
for i in indices:
    print("---"+ str(categories[np.where(np.all(indices==i,axis=1))[0][0]]) + "---")
    for j in i[-15:]:
        print(features[j])
      
 


Predicted: [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0]
Actual: [1, 0, 1, 0, 3, 1, 1, 2, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 2, 0, 0, 1, 3, 1, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 2, 0, 0]
Accuracy: 0.540540540541

---resolves---
gentle
constraint
question
answer
synonymous
force
kc
ball
marriage
line
explanation
noise
miss
bars
file
---elaborates---
root
artist
concentration
bar
distribution
5th
screen tests
tests
screen
negative
andy
conceptual
art
rate
warhol
---social---
vec begin
v_f
tw_f
pmatrix end
slide
begin
nbsp nbsp
urlref4
14
course
end pmatrix
begin pmatrix
nbsp
vec
pmatrix
---requests---
dr
art
dr zeger
zeger
solved
learn
concepts
doing
learn doing
damien
damien hirst
jeff koons
hirst
jeff
koons


In [7]:
print('----------Linear SVM or SGDCLassifier with Hinge Loss------------\n\n')
print(metrics.confusion_matrix(y_val, predicted))
print(metrics.classification_report(y_val, predicted))

# scores = cross_val_score(
#     text_clf, X, df_val['Categories'], cv=2, scoring='f1_macro')

----------Linear SVM or SGDCLassifier with Hinge Loss------------


[[16  3  0  0]
 [ 9  3  0  0]
 [ 3  0  1  0]
 [ 1  1  0  0]]
             precision    recall  f1-score   support

          0       0.55      0.84      0.67        19
          1       0.43      0.25      0.32        12
          2       1.00      0.25      0.40         4
          3       0.00      0.00      0.00         2

avg / total       0.53      0.54      0.49        37



/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
# Gradient Boosted Trees

class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document """

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text)
                }
                for text in posts]

class DenseTransformer(TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

text_clf = Pipeline([
                     ('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
                     
                     ('to_dense', DenseTransformer()),
                     
                     ('clf', GradientBoostingClassifier(random_state=42
                                            )),
])


print('Gradient Boosted Trees -Accuracy and Most Important Features')

text_clf.fit(df_train['Post_text'], y_train,)  

predicted = text_clf.predict(df_val['Post_text'])
#print(df_val['Post_text'],y_val)
print("Predicted: "+ str(predicted.tolist()))
print("Actual: " + str(y_val.tolist()))
print("Accuracy: "+ str(np.mean(predicted == y_val))+ "\n")


#print(np.sort(text_clf.named_steps['clf'].feature_importances_))
indices = np.argsort(text_clf.named_steps['clf'].feature_importances_)[-15:]


features = (text_clf.named_steps['tfidf'].get_feature_names())
print("\nMost important features: \n")
for i in indices:
    print(features[i])
# important_features_dict = {}
# for x,i in enumerate(text_clf.named_steps['clf'].feature_importances_):
#     important_features_dict[x]=i


# important_features_list = sorted(important_features_dict,
#                                  key=important_features_dict.get,
#                                  reverse=True)

# print('Most important features: %s' %important_features_list)

Predicted: [0, 1, 0, 2, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0]
Actual: [1, 0, 1, 0, 3, 1, 1, 2, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 2, 0, 0, 1, 3, 1, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 2, 0, 0]
Accuracy: 0.486486486486


Most important features: 

just
test
art
cool
similar
totally
agree
product
fifth
14
going
warhol
guess
rate
explain


In [9]:
print('----------Gradient Boosted Trees------------\n\n')
print(metrics.confusion_matrix(y_val, predicted))
print(metrics.classification_report(y_val, predicted))



----------Gradient Boosted Trees------------


[[14  4  1  0]
 [ 8  4  0  0]
 [ 2  2  0  0]
 [ 1  1  0  0]]
             precision    recall  f1-score   support

          0       0.56      0.74      0.64        19
          1       0.36      0.33      0.35        12
          2       0.00      0.00      0.00         4
          3       0.00      0.00      0.00         2

avg / total       0.41      0.49      0.44        37



/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
# Random Forest Pipeline

text_clf = Pipeline([
                     ('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
                     
                     ('to_dense', DenseTransformer()),
                     
                     ('clf', RandomForestClassifier(class_weight=class_weights,random_state=42
                                            )),
])


print('Random Forest Pipeline - Accuracy and Most Important Features ')

text_clf.fit(df_train['Post_text'], y_train)  

predicted = text_clf.predict(df_val['Post_text'])
#print(df_val['Post_text'],y_val)
print("Predicted: "+ str(predicted.tolist()))
print("Actual: " + str(y_val.tolist()))
print("Accuracy: "+ str(np.mean(predicted == y_val))+ "\n")

#print(np.sort(text_clf.named_steps['clf'].feature_importances_))
indices = np.argsort(text_clf.named_steps['clf'].feature_importances_)[-15:]
#print(indices)

features = (text_clf.named_steps['tfidf'].get_feature_names())
print("\nMost important features: \n")
for i in indices:
    print(features[i])


Predicted: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
Actual: [1, 0, 1, 0, 3, 1, 1, 2, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 2, 0, 0, 1, 3, 1, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 2, 0, 0]
Accuracy: 0.513513513514


Most important features: 

using
league
film
product
personally
discuss specific
andy
think
questions
derivative crass
urlref4 problem
begin
art
displayed properly
presented lecture


In [11]:
print('----------Random Forest------------\n\n')
print(metrics.confusion_matrix(y_val, predicted))
print(metrics.classification_report(y_val, predicted))

----------Random Forest------------


[[18  1  0  0]
 [10  1  1  0]
 [ 3  1  0  0]
 [ 2  0  0  0]]
             precision    recall  f1-score   support

          0       0.55      0.95      0.69        19
          1       0.33      0.08      0.13        12
          2       0.00      0.00      0.00         4
          3       0.00      0.00      0.00         2

avg / total       0.39      0.51      0.40        37



/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [82]:
'''
Download and Import glove 
'''
print('Download and import glove word embeddings to get the word vectors')

BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))
        

Found 400000 word vectors.


In [83]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

In [84]:
import re
df = pd.read_csv('../data/21total_preprocessed.csv').dropna(subset=['Post_text'])
del df['Task2.1']
df['Categories'] = df['Categories'].factorize()[0]
texts = df['Post_text']
print((texts[0][0]))
labels = df['Categories']

D


In [85]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)

tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print(word_index)

Found 7016 unique tokens.
{'the': 1, 'to': 2, 'of': 3, 'i': 4, 'is': 5, 'a': 6, 'and': 7, 'in': 8, 'that': 9, 'it': 10, 'for': 11, 'this': 12, 'you': 13, 'as': 14, 'be': 15, 'not': 16, 'on': 17, 'but': 18, 'are': 19, 'with': 20, 'have': 21, 'was': 22, '\xa0': 23, 'if': 24, 'or': 25, 'from': 26, 'so': 27, 'what': 28, 'would': 29, 'at': 30, 'we': 31, 'can': 32, 'about': 33, 'an': 34, 'by': 35, 'think': 36, '1': 37, 'his': 38, 'there': 39, 'art': 40, '2': 41, 'warhol': 42, 'all': 43, 'he': 44, 'one': 45, 'urlref': 46, 'more': 47, 'will': 48, 'my': 49, 'same': 50, 'when': 51, 'just': 52, 'me': 53, 'do': 54, 'some': 55, 'like': 56, 'how': 57, 'they': 58, 'math': 59, '0': 60, 'time': 61, 'which': 62, 'question': 63, 'lecture': 64, 'get': 65, 'has': 66, 'am': 67, 'other': 68, 'very': 69, 'then': 70, 'work': 71, 'see': 72, 'your': 73, 'because': 74, 'no': 75, 'course': 76, 'way': 77, 'than': 78, 'also': 79, "i'm": 80, 'know': 81, 'could': 82, 'answer': 83, 'out': 84, 'first': 85, "don't": 86, 

In [86]:
print(len(texts))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

315
Shape of data tensor: (315, 1000)
Shape of label tensor: (315, 4)


In [87]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
print(labels)
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

[[ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 ..., 
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]]


In [88]:
label1 = []
for entry in labels:
    label1.append(np.argmax(entry))
print(np.array(label1).shape)
label1 =np.array(label1)
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

print('Preparing embedding matrix.')
print(y_train.shape)

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

        

(315,)
Preparing embedding matrix.
(252, 4)


In [89]:
import keras.backend as K

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')

from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras.layers import Flatten


# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)

#x = Dense(128,activation='relu')(embedded_sequences)
#x = Flatten()(x)
preds = Dense(4, activation='softmax')(x)

model = Model(sequence_input, preds)
class Metrics(Callback):
    
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
 
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict,average=None)
        _val_recall = recall_score(val_targ, val_predict,average=None)
        _val_precision = precision_score(val_targ, val_predict,average=None)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        return 
    

        
        
metrics1 = Metrics()


model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=12,
          epochs=10,
          validation_data=(x_val, y_val),callbacks=[metrics1])

print((metrics1.val_f1s[-1], metrics1.val_precisions[-1], metrics1.val_recalls[-1]))

Training model.
Train on 252 samples, validate on 63 samples
Epoch 1/10
252/252 [==============================] - 10s 40ms/step - loss: 1.2307 - acc: 0.4643 - val_loss: 1.0312 - val_acc: 0.6349

/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 2/10
252/252 [==============================] - 9s 36ms/step - loss: 1.0050 - acc: 0.5595 - val_loss: 1.0291 - val_acc: 0.6190

In [90]:
class_weights = {
    0: 1.,
    1: 1.,
    2: 5.,
    3: 5.
}

label1 = []
for entry in labels:
    label1.append(np.argmax(entry))
print(np.array(label1).shape)
label1 =np.array(label1)
x_train = data[:-num_validation_samples]
y_train = label1[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = label1[-num_validation_samples:]

#Linear SVM with Glove 
print(embedding_matrix.shape)
from sklearn import linear_model
clf = linear_model.SGDClassifier(class_weight=class_weights)

#labels = np.array(labels)
#print(y_train.shape)
clf.fit(x_train,y_train)

predicted = clf.predict(x_val)
print("Predicted: "+ str(predicted.tolist()))
print("Actual: " + str(y_val.tolist()))
print(len(y_val.tolist()))
print("Accuracy: "+ str(np.mean(predicted == y_val))+ "\n")
#indices = (np.argsort(clf.named_steps['clf'].coef_,axis=1))
#print(indices)
#features = (clf.named_steps['tfidf'].get_feature_names())
# for i in indices:
#     print("---"+ str(categories[np.where(np.all(indices==i,axis=1))[0][0]]) + "---")
#     for j in i[-15:]:
#         print(features[j])


(315,)
(7017, 300)
Predicted: [0, 1, 0, 0, 1, 0, 0, 0, 2, 0, 3, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 1, 2, 0, 3, 2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 1, 3, 0, 0, 2, 0, 2, 0, 3, 0, 2, 2, 0, 2, 0]
Actual: [3, 0, 0, 2, 1, 0, 2, 0, 0, 0, 0, 2, 3, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 1, 2, 1, 0, 3, 2, 0, 0, 3, 0, 0, 2, 0, 2, 0, 0, 0, 2, 1, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0]
63
Accuracy: 0.492063492063



In [91]:
print(metrics.confusion_matrix(y_val, predicted))
print(metrics.classification_report(y_val, predicted))



[[24  4  7  4]
 [ 3  1  2  0]
 [ 8  0  6  0]
 [ 2  0  2  0]]
             precision    recall  f1-score   support

          0       0.65      0.62      0.63        39
          1       0.20      0.17      0.18         6
          2       0.35      0.43      0.39        14
          3       0.00      0.00      0.00         4

avg / total       0.50      0.49      0.49        63

